# Categorizing the questions
- Creating a list of all questions combined
- Creating mutiple lists of different types of categories a question might belong to
- Calculating the total number of questions belonging to different category

In [1]:
#getting all the imports
import numpy as np
import pandas as pd
import re
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
from subprocess import check_output

%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

In [2]:
import os
#getting current working directory
b = os.getcwd()

#reading the csv file
#df = pd.read_csv(b+"/"+"Data/vehicle_collisions.csv", parse_dates=['DATE'], usecols=[0,1,3]) #vehicle_collisions.csv, , delim_whitespace=True, , header=None

df = pd.read_csv(b+"/"+"Data/questions.csv", usecols=[3,4,5]).fillna("")
df.head() 

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [3]:
#removing the special characters from the questions
#df['question1']=df['question1'].map(lambda x: re.sub(r'\W+', ' ', x))
#df['question2']=df['question2'].map(lambda x: re.sub(r'\W+', ' ', x))

In [25]:
#creating the list of types of questions
Explaination = ['what']
Reasoning = ['why', 'how']
Categorical = ['which']
Numeric = ['how much','how many']
Location = ['where']
timeline = ['when']
Y_N = ["does","should"]

In [26]:
#creating a subset of dataframes based on the similarity of questions
df1 = df[df['is_duplicate']== 0]
df2 = df[df['is_duplicate']== 1]

In [27]:
#resetting the indexes of both similar and dissmilar dataframes
df1 = df1.reset_index()
df2 = df2.reset_index()

In [28]:
#creating one single list of all the questions
qs1 = []

#for non duplicate questions add both the questions to the list
for i in range(0, len(df1)):
    qs1.append(df1['question1'][i])
    qs1.append(df1['question2'][i])

#for duplicate questions add just one question
for i in range(0,len(df2)):
    qs1.append(df2['question1'][i])

In [29]:
#converting all the words in df2 list to lowercase
qs1_lower = []
for questions in qs1:
    qs1_lower.append(questions.lower())

In [31]:
##### Runs for longer time #######

# Offsetting the warning
#pd.options.mode.chained_assignment = None

from fuzzywuzzy import fuzz


#checking the matching index between the questions and diffrent types of questions list
count_exp = []
count_reasoning = []
count_categorical = []
count_numeric = []
count_location = []
count_timeline = []
count_y_n = []

#checking the question against all categories
for qs in qs1_lower:
    count_exp.append(fuzz.token_set_ratio(qs,Explaination))
    count_reasoning.append(fuzz.token_set_ratio(qs,Reasoning))
    count_categorical.append(fuzz.token_set_ratio(qs,Categorical))
    count_numeric.append(fuzz.token_set_ratio(qs,Numeric))
    count_location.append(fuzz.token_set_ratio(qs,Location))
    count_timeline.append(fuzz.token_set_ratio(qs,timeline))
    count_y_n.append(fuzz.token_set_ratio(qs,Y_N))

In [32]:
len(count_categorical)

659396

In [34]:
#converting all the lists to Dataframe
qs1_lower = pd.DataFrame(qs1_lower)
count_exp = pd.DataFrame(count_exp)
count_categorical = pd.DataFrame(count_categorical)
count_location = pd.DataFrame(count_location)
count_numeric = pd.DataFrame(count_numeric)
count_reasoning = pd.DataFrame(count_reasoning)
count_timeline = pd.DataFrame(count_timeline)
count_y_n = pd.DataFrame(count_y_n)

In [37]:
#merging the dataframes to a single dataframe
final_df = pd.concat([qs1_lower, count_categorical,count_exp,count_location,count_numeric,count_reasoning,count_timeline,count_y_n], axis=1)
final_df.columns = ['questions','categorical %','explanaitive %','location based %','numeric %',
                   'reasoning %','timeline %','Yes or No %']

In [38]:
final_df.head()

,questions,categorical %,explanaitive %,location based %,numeric %,reasoning %,timeline %,Yes or No %
0,what is the step by step guide to invest in sh...,6,100,10,14,16,7,21
1,what is the step by step guide to invest in sh...,7,100,11,16,17,7,23
2,what is the story of kohinoor (koh-i-noor) dia...,11,100,8,16,15,8,17
3,what would happen if the indian government sto...,7,100,7,13,9,5,22
4,how can i increase the speed of my internet co...,8,5,8,38,60,11,10


In [39]:
#creating empty weighted dataframes to calculate the weight of each question in each category
weight_exp = pd.DataFrame().reindex_like(count_exp)
weight_reasoning = pd.DataFrame().reindex_like(count_exp)
weight_categorical = pd.DataFrame().reindex_like(count_exp)
weight_numeric = pd.DataFrame().reindex_like(count_exp)
weight_location = pd.DataFrame().reindex_like(count_exp)
weight_timeline = pd.DataFrame().reindex_like(count_exp)
weight_y_n = pd.DataFrame().reindex_like(count_exp)


In [42]:
#merging the weighted dataframes to the the final df
final_df2 = pd.concat([final_df, weight_exp,weight_reasoning,weight_categorical,weight_numeric,weight_location,weight_timeline,weight_y_n], axis=1)
final_df2.columns = ['questions','categorical %','explanaitive %','location based %','numeric %',
                   'reasoning %','timeline %','Yes or No %', 'Explaination','Reasoning','Categorical',
                    'Numeric','Location','Timeline','Y_N']



In [43]:
final_df2.head()

,questions,categorical %,explanaitive %,location based %,numeric %,reasoning %,timeline %,Yes or No %,Explaination,Reasoning,Categorical,Numeric,Location,Timeline,Y_N
0,what is the step by step guide to invest in sh...,6,100,10,14,16,7,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,what is the step by step guide to invest in sh...,7,100,11,16,17,7,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,what is the story of kohinoor (koh-i-noor) dia...,11,100,8,16,15,8,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,what would happen if the indian government sto...,7,100,7,13,9,5,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,how can i increase the speed of my internet co...,8,5,8,38,60,11,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
# Offsetting the warning
pd.options.mode.chained_assignment = None


#assigning weights to different categories
#if the weight value is more than or equal to 60% the weight in that category is 1 else 0
#one question can belong to multiple categories

for i in range(0,len(final_df2)):
    if (final_df2['categorical %'][i]) >59:
        final_df2['Categorical'][i] = 1
    else:
        final_df2['Categorical'][i] = 0
    
    if (final_df2['explanaitive %'][i]) >59:
        final_df2['Explaination'][i] = 1
    else:
        final_df2['Explaination'][i] = 0
    
    if (final_df2['location based %'][i]) >59:
        final_df2['Location'][i] = 1
    else:
        final_df2['Location'][i] = 0
        
    if (final_df2['numeric %'][i]) >59:
        final_df2['Numeric'][i] = 1
    else:
        final_df2['Numeric'][i] = 0
        
    if (final_df2['reasoning %'][i]) >59:
        final_df2['Reasoning'][i] = 1
    else:
        final_df2['Reasoning'][i] = 0
        
    if (final_df2['timeline %'][i]) >59:
        final_df2['Timeline'][i] = 1
    else:
        final_df2['Timeline'][i] = 0
        
    if (final_df2['Yes or No %'][i]) >59:
        final_df2['Y_N'][i] = 1
    else:
        final_df2['Y_N'][i] = 0
        

In [45]:
final_df2.head(2)

,questions,categorical %,explanaitive %,location based %,numeric %,reasoning %,timeline %,Yes or No %,Explaination,Reasoning,Categorical,Numeric,Location,Timeline,Y_N
0,what is the step by step guide to invest in sh...,6,100,10,14,16,7,21,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,what is the step by step guide to invest in sh...,7,100,11,16,17,7,23,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
#copy_df = final_df2.copy()

In [48]:
#create a dataframe with just the weights
result_df = final_df2.loc[:, 'Explaination' : 'Y_N']

In [49]:
result_df.head()

,Explaination,Reasoning,Categorical,Numeric,Location,Timeline,Y_N
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [50]:
#get the sum of all the weights
sum_df = result_df.sum().reset_index()

In [51]:
#renaming the names of the columns
sum_df.columns = ['type of weights', 'sum']

In [52]:
#sorting the result dataframe in descending order
sum_df = sum_df.sort_values(['sum'], ascending=[False])

sum_df

,type of weights,sum
0,Explaination,258087.0
1,Reasoning,236727.0
2,Categorical,34977.0
6,Y_N,32086.0
5,Timeline,22274.0
3,Numeric,15648.0
4,Location,14627.0
